In [1]:
# Import libraries

import json
import pandas as pd
from elasticsearch import Elasticsearch, exceptions
from tqdm.auto import tqdm
import minsearch

c:\Users\ADMIN\.virtualenvs\LLMs-Course-DataTalksClub-n7mUlEcu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Read JSON document with generated ids

with open('..\data\data-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [3]:
# Load generated ground truth data

df_questions = pd.read_csv('..\data\ground-truth-retrieval.csv')
ground_truth = df_questions.to_dict(orient='records')

In [4]:
ground_truth[100]

{'id': '32512413',
 'question': 'What are the tests used to diagnose anal cancer?'}

#### **Elastic Search**

In [5]:
es = Elasticsearch('http://localhost:9200') 

# Define index name and settings
index_name = "health-questions"

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "question": {"type": "text"},
            "answer": {"type": "text"},
            "source": {"type": "text"},
            "focus_area": {"type": "text"}
        }
    }
}

In [9]:
# Create the index if it doesn't exist

try:
    if not es.indices.exists(index=index_name):
        es.indices.create(index=index_name, body=index_settings)
        print("Index created successfully")
    else:
        print("Index already exists")
except exceptions.ConnectionError as e:
    print(f"Failed to create index: {e}")

Index already exists


In [10]:
# Index the documents
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|██████████| 292/292 [01:11<00:00,  4.07it/s]


In [11]:
# Define a function that retrieves documents and matches user queries

def elastic_search(query, max_results=5):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "answer^3", 'focus_area'],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

#### **Minsearch**

In [18]:
index = minsearch.Index(
    text_fields=["question", "answer", "focus_area"],
    keyword_fields=["id"]
)

index.fit(documents)

In [23]:
def minsearch_search(query):
    boost = {'question': 3.0, 'answer' : 3.0, 'focus_area' : 1.0}

    results = index.search(
        query=query,
        boost_dict=boost,
        num_results=5
    )

    return results

#### Evaluation metrics :
  - *Hit Rate (recall)*
  - *Mean Reciprocal Rank (mrr)*

In [13]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [25]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank]:  # If relevant document found
                total_score += 1 / (rank + 1)
                break  # Stop after finding the first relevant document

    return total_score / len(relevance_total)

#### Retrieval Evaluation

In [16]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [26]:
# Elastic search
evaluate(ground_truth, lambda q: elastic_search(q['question']))

100%|██████████| 1460/1460 [01:31<00:00, 15.89it/s]


{'hit_rate': 0.9342465753424658, 'mrr': 0.867705479452056}

In [27]:
# Minsearch
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

100%|██████████| 1460/1460 [00:10<00:00, 141.60it/s]


{'hit_rate': 0.9671232876712329, 'mrr': 0.930673515981735}

Minsearch produces better results than Elastic Search